## <center> Scraping FirstMonday.com</center>

Prerequisites:
* Make sure to place chromedriver.exe in the same directory as your code, in "\chromedriver-win64" subfolder.

In [1]:
%run 00_lib_sqlwriter.ipynb

In [2]:
# ##### Web Scraper Using Selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

import pandas as pd
import re
from datetime import datetime, timezone
import time
import os

class FirstMondayBot:
    """
    A bot to automate the scraping of journals using Selenium.
    """
    def __init__(self, driver, mysql_writer):
        # Initializes the bot with a Selenium WebDriver instance and a MySQLWriter instance.
        self.driver = driver
        self.mysql_writer = mysql_writer

    def scrape_archives_urls (self, base_url, max_pages=1):
        # Scrapes archives URLs and stores them in the 'archives' table.

        current_page = 0

        try:
            # Navigate to the base URL
            self.driver.get(base_url)
            time.sleep(2)

            while current_page <= max_pages:
                current_page += 1
                print(f"Scraping page {current_page}...")

                # Wait for the issues_archive to be visible
                WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "issues_archive"))
                )

                # Find and extract links
                struct_item_container = self.driver.find_element(By.CLASS_NAME, "issues_archive")
                post_links = struct_item_container.find_elements(By.XPATH, ".//a[@href]")

                # Collect unique archives URLs from the current page
                base_urls = set()
                for link in post_links:
                    # archive_url, volume_number, title, publication_date, editor, import_date
                    url = link.get_attribute("href")
                    volume_number = ''
                    title = ''
                    publication_date = '1900-01-01'
                    editor = ''
                    base_urls.add((url, volume_number, title, publication_date, editor, datetime.now(), 'PENDING'))

                # Convert the set of URLs to a DataFrame
                if base_urls:
                    archives_df = pd.DataFrame(list(base_urls), columns=['archive_url','volume_number', 'title', 'publication_date', 'editor', 'import_date', 'status'])
                    
                    # Insert into the database
                    self.mysql_writer.insert_archives(archives_df)
                    print(f"Inserted {len(base_urls)} archives URLs from page {current_page}.")

                # Attempt to click the "next" button to proceed to the next page
                try:
                    next_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'next')]"))
                    )
                    self.driver.execute_script("arguments[0].click();", next_button)

                    # Wait for the new page content to load by checking the staleness of the previous container
                    WebDriverWait(self.driver, 3).until(EC.staleness_of(struct_item_container))
                    
                except Exception as e:
                    print("No more pages or error clicking the 'next' button:", e)
                    break
                
        except Exception as e:
            print(f"An error occurred while scraping archives: {e}")

    def scrape_archives (self):
        try:
            df = self.mysql_writer.read_table_archives()
            
            for _, row in df.iterrows():
                archive_url = row['archive_url']
                print('Scraping archive_url:', archive_url)

                self.driver.get(archive_url)
                time.sleep(2)

                bsObj = BeautifulSoup(self.driver.page_source, 'lxml') # Get the page source and parse it with Beautiful Soup

                volume_number = ''
                title = ''
                publication_date = '1900-01-01'
                editor = ''

                volume_number = bsObj.find("h1").getText().strip()
                # print('volume_number: ', volume_number)

                # Find the <div> with class "description"
                description_div = bsObj.find("div", {"class": "description"})
                # print('description:', description_div)

                if description_div:
                    # Extract title and editor
                    paragraph = description_div.find("p")
                    # print('paragraph:', paragraph)
                    
                    if paragraph:
                        try:
                            description = paragraph.get_text(separator='\n').strip()
                            lines = description.split('\n')
                            # print('lines:', lines)

                            title = lines[0].strip()
                            # print('title:', title)

                            # Check if lines[1] contains "edited by" or "co-edited by" 
                            if any(keyword in description.lower() for keyword in ["edited by", "co-edited by"]): 
                                editor = lines[1].strip() 
                                editor = re.sub(r'Edited by|Co-edited by', '', editor, flags=re.IGNORECASE).strip() 
                                # print('editor:', editor)

                        except Exception as e:
                            print("An error occurred:", e)
                            continue

                published_div = bsObj.find("div", {"class": "published"})
                if published_div:
                    publication_date = published_div.find("span", {"class": "value"}).text.strip()
                    # print('publication_date:', publication_date)

                base_urls = set()
                base_urls.add((archive_url, volume_number, title, publication_date, editor, datetime.now(), 'COMPLETED'))
            
                # Update archives table
                archives_df = pd.DataFrame(list(base_urls), columns=['archive_url', 'volume_number', 'title', 'publication_date', 'editor', 'import_date', 'status'])
                self.mysql_writer.update_archives(archives_df)

        except Exception as e:
            print("An error occurred:", e)

    def scrape_articles_urls (self):
        try:
            df = self.mysql_writer.read_table_archives()
            article_urls = set()

            for _, row in df.iterrows():
                archive_url = row['archive_url']

                self.driver.get(archive_url)
                time.sleep(3)

                bsObj = BeautifulSoup(self.driver.page_source, 'lxml') # Get the page source and parse it with Beautiful Soup

                # article_url, title, doi, publication_date, author, archive_url, import_date
                article_url = ''
                title = ''
                doi = ''
                publication_date = '1900-01-01'
                author = ''
                abstract = ''
                content_url = ''

                articles = bsObj.find_all("h3", {"class": "title"})
                # print (articles)
                for article in articles:
                    a_tag = article.find('a')
                    if a_tag:
                        article_url = a_tag['href']
                        # print('article_url: ', article_url)
                        article_urls.add((article_url, title, doi, publication_date, author, abstract, archive_url, content_url, datetime.now(), 'PENDING'))

                articles_df = pd.DataFrame(list(article_urls), columns=['article_url', 'title', 'doi', 'publication_date', 'author', 'abstract', 'archive_url', 'content_url', 'import_date', 'status'])
                self.mysql_writer.insert_articles(articles_df)
            # print(f"Inserted article URL: {(article_urls)}")

        except Exception as e:
            print("An error occurred:", e)

    def scrape_articles (self):
        try:
            df = self.mysql_writer.read_table_articles(status="PENDING")
            article_urls = set()

            for _, row in df.iterrows():
                count=+1
                article_url = row['article_url']
                print('----------------Scraping article_url:', article_url, '----------------')

                archive_url = row['archive_url']
                # print('archive_url:', archive_url)

                self.driver.get(article_url)
                time.sleep(2)

                bsObj = BeautifulSoup(self.driver.page_source, 'lxml') # Get the page source and parse it with Beautiful Soup
                
                # article_url, title, doi, publication_date, author, abstract, archive_url, import_date
                title = ''
                doi = ''
                publication_date = '1900-01-01'
                author = ''
                abstract = ''
                content_url = ''
                error = ''

                if bsObj.find("div", {"class": "error-code"}):
                    error = (bsObj.find("div", {"class": "error-code"}).get_text().strip())
                    print ('error:', error)
                    if error.lower() == 'http error 500':
                        continue

                # Find the <div> with class "description"
                title = bsObj.find("h1", {"class": "page_title"})
                if title: title = title.get_text().strip()
                print('title:', title)

                doi_section = bsObj.find("section", {"class": "item doi"})
                if doi_section: doi = doi_section.find('a').get('href')
                print('doi:', doi)
                
                date_section = bsObj.find("div", {"class": "item published"})
                if date_section: date_section = date_section.find("span")
                if date_section: publication_date = date_section.get_text().strip()
                publication_date = publication_date[:10]
                print('publication_date:', publication_date)

                author_section = bsObj.find("section", {"class": "item authors"})
                if author_section: author_section = author_section.find_all("span",{"class":"name"})
                if author_section:  author = [author.get_text().strip() for author in author_section]
                author = ', '.join(author)
                print('author:', author)

                abstract_section = bsObj.find("section", {"class": "item abstract"})
                if abstract_section:
                    abstract = abstract_section.find("p")
                    if  abstract:
                        abstract = abstract.get_text().strip()
                    else:
                        abstract = abstract_section.get_text(separator=" ", strip=True).strip()
                        abstract = re.sub(r'^\babstract\b', '', abstract, flags=re.IGNORECASE).replace('\n', '')
                print('abstract:', abstract)

                a_tag = bsObj.find("a", {"class": "obj_galley_link file"})
                if a_tag: content_url = a_tag.get('href')
                print('content_url:', content_url)

                article_urls.add((article_url, title, doi, publication_date, author, abstract, archive_url, content_url, datetime.now(), 'COMPLETED'))

                # Update archives table                
                articles_df = pd.DataFrame(list(article_urls), columns=['article_url', 'title', 'doi', 'publication_date', 'author', 'abstract', 'archive_url', 'content_url', 'import_date', 'status'])
                self.mysql_writer.update_articles(articles_df)
                time.sleep(3)

        except Exception as e:
            print("An error occurred:", e)

    def scrape_contents (self):
        try:
            df = self.mysql_writer.read_table_articles_wo_contents ()
            content_urls = set()

            for _, row in df.iterrows():
                content_url = row['content_url']
                print('----------------Scraping content_url:', content_url, '----------------')

                content_url = row['content_url']
                print('content_url:', content_url)

                self.driver.get(content_url)
                time.sleep(3)

                bsObj = BeautifulSoup(self.driver.page_source, 'lxml') # Get the page source and parse it with Beautiful Soup
                print(bsObj.get_text())

                paragraphs = bsObj.find("body",{"class": "pkp_page_article pkp_op_view"})
                print('content:', paragraphs.get_text())
                # if contents: content = [content.get_text().strip() for content in contents]
                # print('content:', content)

                # content_urls.add((content_url, content))

                # # Update contents table                
                # contents_df = pd.DataFrame(list(content_url), columns=['content_url', 'content'])
                # self.mysql_writer.insert_contents (contents_df)
                # time.sleep(5)

        except Exception as e:
            print("An error occurred:", e)      

    def close(self):
        # Closes the Selenium WebDriver instance.
        self.driver.quit()

In [ ]:
# ##### Main Execution: Initializing and Running the Scraper

import os
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Setup Chrome options for Selenium WebDriver
chrome_options = Options()
# chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/111.0.0.0 Safari/537.36")

# Initialize the WebDriver
chrome_driver_path = os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe")
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Define login parameters
URL = "https://firstmonday.org/ojs/index.php/fm/issue/archive"

# MySQL Connection Details
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "root"
DB_NAME = "fmdb"

# Initialize the MySQLWriter instance
mysql_writer = MySQLWriter(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)

# Create an instance of FirstMondayBot
bot = FirstMondayBot(driver, mysql_writer)

try:
    # Scrape archives' URLs and store them in the 'archives' table
    BASE_FORUM_URL = "https://firstmonday.org/ojs/index.php/fm/issue/archive/"
    MAX_PAGES = 50  # Max page 50
    # print("1. Inserting archive URLs...")
    # bot.scrape_archives_urls (BASE_FORUM_URL, MAX_PAGES)

    # # Scrape archives and store them in the 'archives' table
    # print("2. Updating archives...")
    # bot.scrape_archives()

    print("3. Inserting article URLs...")
    bot.scrape_articles_urls()

    print("4. Updating articles...")
    bot.scrape_articles()

    # print("5. Inserting html contents...")
    # bot.scrape_contents()

except Exception as e:
    print("An error occurred during scraping:", e)

finally:
    # Close the WebDriver and the MySQL connection
    bot.close()
    mysql_writer.close_connection()

title: SURVIVORS: Archiving the history of bulletin board systems and the AIDS crisis
doi: https://doi.org/10.5210/fm.v25i10.10290
publication_date: 2020-09-22
author: Kathryn Brewster, Bonnie Ruberg
abstract: The history of the Internet and the history of the HIV/AIDS crisis are fundamentally intertwined. Because of the precarious nature of primary early Internet materials, however, documentation that reflects this relationship is limited. Here, we present and analyse an important document that offers considerable insight in this area: a full printout of the bulletin board system (BBS) discussion group “SURVIVORS.” Run by David Charnow, SURVIVORS operated as an “electronic support group” for members living with HIV/AIDS from 1987 to 1990. These dates represent a period of overlap between both the AIDS crisis in America and the use of BBSs as a predecessor to contemporary Internet technologies. The contents of SURVIVORS were printed by Charnow before his death in 1990 and later donated